In [1]:
import pandas as pd
import json
import os
import ast 

import numpy as np
from tqdm import tqdm
import chardet
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px

from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import textblob
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import math
from nltk import bigrams,trigrams,ngrams
from collections import Counter
from sklearn.metrics import confusion_matrix
from nltk import everygrams, word_tokenize
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_confusion_matrix

# Deal with business data

In [2]:
business = pd.read_json('./yelp_dataset_2022/yelp_dataset_2022/business.json',lines = True)
IL_business = business[business['state'] == 'CA']
IL_business = IL_business.reset_index()

In [3]:
for i in range(len(IL_business)):
    cur_attr = IL_business['attributes'][i]
    if bool(cur_attr) == False:
        continue
    for key in cur_attr.keys():
        IL_business.loc[i,key] = cur_attr[key]


def extract_info (df, feature):
    for i in range(len(df)):
        try:
            cur_attr = df[feature][i]
            cur_attr = ast.literal_eval(cur_attr)
            for key in cur_attr.keys():
                df.loc[i,key] = cur_attr[key]
        except: 
            continue

            
def extract_info_hours (df, feature):
    for i in range(len(df)):
        try:
            cur_attr = df[feature][i]
            #cur_attr = ast.literal_eval(cur_attr)
            for key in cur_attr.keys():
                df.loc[i,key] = cur_attr[key]
        except: 
            continue
            
extract_info(IL_business,'BusinessParking')    
extract_info(IL_business,'GoodForMeal')     
extract_info(IL_business,'Ambience')     
extract_info_hours(IL_business,'hours')     
     
        
IL_business = IL_business.drop(columns=['attributes','BusinessParking','GoodForMeal','Ambience'])
IL_business = IL_business.rename({'stars':'average_stars'},axis = 1)

In [4]:
Asian_keyword = ['Chinese','Korean','Japanese','Thai','Vietnamese','Asian','Sushi']
Asian_list = []
for i in range(len(IL_business)):
    cur_cat = IL_business.loc[i,'categories']
    try: 
        cur_cat = cur_cat.split(',')
        if any(item in cur_cat for item in Asian_keyword):
            Asian_list.append(IL_business.loc[i,'name'])
    except: continue
        
IL_Asian_food = IL_business[IL_business.name.str.contains('|'.join(Asian_list))]
#IL_Asian_food = IL_Asian_food.reset_index(drop=True)

# Turn Monday-Sunday working hour to length

In [6]:
def convert_hour(time):
    result = 0
    if len(time) == 5:
        if time[-2] == '00':
            result = int(time[:-3])
        elif int(time[-2]) > 15:
            result = int(time[:-3])+1
        else:
            result = int(time[:-3])
    elif len(time) == 4:
        if time[-1] == '0':
            result = int(time[:-2])
        elif int(time[-1]) > 15:
            result = int(time[:-2])+1
        else:
            result = int(time[:-2])
    else:
        if time[-1] == '0':
            result = int(time[:-2])
        elif int(time[-1]) > 15:
            result = int(time[:-2])+1
        else:
            result = int(time[:-2])
    return result
            

def calculate_total_hour(feature, new_feature):
    for i in range(len(IL_Asian_food)):
        try:
            cur_schedule =  IL_Asian_food.loc[i,feature]
            ls = cur_schedule.split('-')
            if ls[0] == ls[1]:
                IL_Asian_food.loc[i,new_feature] = 24
            else:
                work_hour = convert_hour(ls[1])-convert_hour(ls[0])
                IL_Asian_food.loc[i,new_feature] =work_hour
        except: 
            continue
        

calculate_total_hour('Monday','Monday_working_length')
calculate_total_hour('Tuesday','Tuesday_working_length')
calculate_total_hour('Wednesday','Wednesday_working_length')
calculate_total_hour('Thursday','Thursday_working_length')
calculate_total_hour('Friday','Friday_working_length')
calculate_total_hour('Saturday','Saturday_working_length')
calculate_total_hour('Sunday','Sunday_working_length')

IL_Asian_food = IL_Asian_food.drop(columns=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']).reset_index(drop = True)

# Read and combine reivew with business

In [7]:
review = pd.read_csv('review.csv')
review = review.rename({'stars':'comment_star'},axis = 1)
review.head()

,review_id,user_id,business_id,comment_star,date,text
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,2018-07-07 22:09:11,"If you decide to eat here, just be aware it is..."
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,2012-01-03 15:28:18,I've taken a lot of spin classes over the year...
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,2014-02-05 20:30:30,Family diner. Had the buffet. Eclectic assortm...
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,2015-01-04 00:01:03,"Wow! Yummy, different, delicious. Our favo..."
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,2017-01-14 20:54:15,Cute interior and owner (?) gave us tour of up...


In [9]:
frames = [IL_Asian_food, review]
IL_Asian_business_review = pd.merge(review, IL_Asian_food,left_on='business_id',right_on = 'business_id', how = 'right' ).reset_index(drop=True)
IL_Asian_business_review.to_csv('IL_Asian_business_review.csv',index = False)

# EDA

In [ ]:
cur_open = IL_Asian_business_review[IL_Asian_business_review['is_open'] == 1]
cur_closed = IL_Asian_business_review[IL_Asian_business_review['is_open'] == 0]

In [ ]:
#plt.figure(figsize = (12,8))
sns.countplot(IL_Asian_food['average_stars'])
plt.xlabel('Average stars')
plt.ylabel('Total count')
plt.show()

# Interactive map

In [ ]:
has_parking = IL_Asian_food[IL_Asian_food['lot'] == True]
no_parking = IL_Asian_food[IL_Asian_food['lot'] == False]

In [ ]:
color_scale = [(0, 'orange'), (1,'red')]
fig = px.scatter_mapbox(IL_Asian_food, 
                        lat="latitude", 
                        lon="longitude", 
                        hover_name="name", 
                        hover_data=["average_stars",'address'],
                        color="lot",
                        color_continuous_scale=color_scale,
                        size="average_stars",
                        zoom=8, 
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:

color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(IL_Asian_food, 
                        lat="latitude", 
                        lon="longitude", 
                        hover_name="name", 
                        hover_data=["average_stars"],
                        color="average_stars",
                        color_continuous_scale=color_scale,
                        size="average_stars",
                        zoom=8, 
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
IL_Asian_business_review['text'].head()[0]

In [ ]:
IL_Asian_business_review['text_cleaned'].head()[0]

# NLP

In [ ]:
#Remove stopwords
lemm = WordNetLemmatizer()
#stop = set(stopwords.words('english'))
stop = stopwords.words('english')
IL_Asian_business_review['text_word_list'] = IL_Asian_business_review['text'].str.split()
IL_Asian_business_review['text_no_stopwords'] = IL_Asian_business_review['text_word_list'].apply(lambda x : [re.sub(r'[.,"\'-?:!;]', '', i) for i in x])
IL_Asian_business_review['text_no_stopwords'] = IL_Asian_business_review['text_no_stopwords'].apply(lambda x : [re.sub(r'\<a href', ' ', i) for i in x])
IL_Asian_business_review['text_no_stopwords'] = IL_Asian_business_review['text_no_stopwords'].apply(lambda x : [re.sub(r'&amp;', '', i) for i in x])
IL_Asian_business_review['text_no_stopwords'] = IL_Asian_business_review['text_no_stopwords'].apply(lambda x : [re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', i) for i in x])
IL_Asian_business_review['text_no_stopwords'] = IL_Asian_business_review['text_no_stopwords'].apply(lambda x : [re.sub(r'<br />', ' ', i) for i in x])
IL_Asian_business_review['text_no_stopwords'] = IL_Asian_business_review['text_no_stopwords'].apply(lambda x : [re.sub(r'\'', ' ', i) for i in x])
IL_Asian_business_review['text_no_stopwords'] = IL_Asian_business_review['text_no_stopwords'].apply(lambda x : [i for i in x if i not in stop])
IL_Asian_business_review['text_no_stopwords'] = IL_Asian_business_review['text_no_stopwords'].apply(lambda x: [item.lower() for item in x])
IL_Asian_business_review['text_cleaned'] = [' '.join(map(str,l)) for l in IL_Asian_business_review.text_no_stopwords]

def lemmatize_text(text):
    return [lemm.lemmatize(word) for word in nltk.WordPunctTokenizer().tokenize(text)]
IL_Asian_business_review['text_cleaned'] = IL_Asian_business_review.text_cleaned.apply(lemmatize_text)

In [ ]:
#Categorize each sentiment
def categorize_sentiment(row):
    if row['comment_star'] > 3:
        return 'Positive'
    elif row['comment_star'] == 3:
        return 'Neutral'
    elif row['comment_star'] < 3:
        return 'Negative'
IL_Asian_business_review['sentiment'] = IL_Asian_business_review.apply(lambda row : categorize_sentiment(row), axis = 1)

In [ ]:
#Plot word cloud
def word_cloud(df, sentiment):
    new_df= df[df['sentiment'] == sentiment].reset_index(drop=True)
    wc = ''
    for i in range(len(new_df)):
        for word in new_df.loc[i,'text_cleaned']:
            wc += ''.join(word)+' '

    wordcloud = WordCloud(width = 800, height = 800,
                          background_color ='white',min_font_size = 10,
                         collocation_threshold = 100, min_word_length=3).generate(wc)

    plt.figure(figsize = (4, 4), facecolor = None)
    plt.imshow(wordcloud)
    # plt.title(sentiment)
    plt.axis("off")
    plt.tight_layout(pad = 0)
    
word_cloud(IL_Asian_business_review,'Positive')   
word_cloud(IL_Asian_business_review,'Negative')
word_cloud(IL_Asian_business_review,'Neutral')

# n_gram plot top-10

In [ ]:
def generate_N_grams(text,ngram):
  temp=zip(*[text[i:] for i in range(0,ngram)])
  ans=[' '.join(ngram) for ngram in temp]
  return ans

def run_ngram(training_set, gram_num):
    #Add positive word and negative word count into their own dictionary respectivly
    positive_dict = defaultdict(int)
    negative_dict = defaultdict(int)

    for text in training_set[training_set.sentiment == 'Positive'].text_cleaned:
        for word in generate_N_grams(text,gram_num):
            positive_dict[word] += 1

    for text in training_set[training_set.sentiment == 'Negative'].text_cleaned:
        for word in generate_N_grams(text,gram_num):
            negative_dict[word] += 1

    #Turn the two dictionary into dataframe and find the most frequent words
    df_positive = pd.DataFrame(sorted(positive_dict.items(), key = lambda x : x[1], reverse= True))
    df_negative = pd.DataFrame(sorted(negative_dict.items(), key = lambda x : x[1], reverse= True))

    
    #Bar plot for top ten positive words
    plt.figure(figsize=(6,2))
    plt.subplots_adjust(top = 6, bottom=3,left = 5, right = 8)
    plt.subplot(1,2,1)
    plt.bar(df_positive[0][:10], df_positive[1][:10], color = 'green', width = 0.3)
    if gram_num > 1:
        plt.xticks(rotation = 45)
    plt.ylabel("Count")
    plt.title("Top 10 positive")
    
    
    #Bar plot for top ten positive words
    plt.subplot(1,2,2)
    plt.bar(df_negative[0][:10], df_negative[1][:10], color = 'red', width = 0.3)
    if gram_num > 1:
        plt.xticks(rotation = 45)
    plt.ylabel("Count")
    plt.title("Top 10 negative")

In [ ]:
run_ngram(IL_Asian_business_review,2)

In [ ]:
run_ngram(IL_Asian_business_review,3)

# TFIDF

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,4))
IL_Asian_business_review['text_new'] = [' '.join(map(str,l)) for l in IL_Asian_business_review.text_cleaned]
transformed = tfidf.fit_transform(pd.Series(IL_Asian_business_review.text_new))

tfidf_test_df = pd.DataFrame(transformed[0].T.todense(),index=tfidf.get_feature_names(), columns=["TF-IDF"])
tfidf_test_df = tfidf_test_df.sort_values('TF-IDF', ascending=False)

tfidf_test_df